# PROGETTO BASE DI DATI BREDARIOL FRANCESCO

Sviluppiamo all'interno di questo notebook una semplice interfaccia che permetta di utilizzare la base di dati preparate in vista dell'esame di Base di Dati.

Per la precisione creeremo un semplice ambiente che permette di simulare il passare degli anni e dunque cambiare lo stato del database. L'ambiente permetterà di effettuare le operazioni principali definite nel progetto (operazioni A, B, C, D, E) in maniera personalizzata.

## SISTEMA TRIBUTARIO IMPERO GALATTICO DI AGOSTINO MDI

Anziché usare semplicemente cursor.execute() definiamo questa versione più sicura che prevede la gestione degli errori ed il commit automatico della query tramite prepared statement.

In [ ]:
import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(host="localhost", user="root", password="AIDA2023!!aaa", database="STI")

In [ ]:
def execute_query(db, query, data=None):
    cursor = db.cursor()
    try:
        if data:
            cursor.execute(query, data)
        else:
            cursor.execute(query)
        db.commit()
    except Error as e:
        if str(e) != 'Unread result found':
            print(f"The error '{e}' occurred")
    try:
        res = cursor.fetchall()
        cursor.close()
        return res
    except Error as e:
        return None

L'anno corrente per il db è il 2026. Nel 2027 avverranno nuove elezioni. Le elezioni avvengono ogni 3 anni, dunque per l'esattezza ogni volta che YYYY%3 è uguale a 2. Le emissioni dei tributi avvengono il primo di gennaio di ogni anno.

**Nota** 

Settiamo l'anno corrente in realtà come l'anno del pagamento avvenuto più tardi tra tutti i pagamenti presenti così che se venisse eseguito il notebook python più volte senza risettare tramite script MySQL il database la logica resterebbe intatta.

In [ ]:
query = "select year(max(dataDiEmissione)) from tributi group by DataDiEmissione;"
res = execute_query(mydb, query)
CURRENT_YEAR = int(res[0][0])

Definiamo la funzione che simula il passare di un anno.

In [ ]:
def one_year_later(db):
    global CURRENT_YEAR
    CURRENT_YEAR = CURRENT_YEAR + 1
    data = (f'{CURRENT_YEAR}-01-01',)
    execute_query(db, "CALL batchtributi(%s)", data)
    if CURRENT_YEAR%3 == 2:
        new_election(db)
    paga_tributi(db)
    aggiorna_mondi(db)
    print(f"ora siamo nel {CURRENT_YEAR}")
    return 0

Andiamo a definire una funzione che crei per ogni distretto un nuovo Console.

**Nota** 

Viene importata la libreria faker per poter generare nomi casuali facilmente per i Consoli. Qualora si volesse effettivamente vedere il funzionamento del programma, la documentazione per il download della libreria è [qui](https://faker.readthedocs.io/en/master/).

In [ ]:
from faker import Faker
import datetime
from datetime import date, timedelta
import random

def contatto_random():
    return f'+{random.randint(30, 40)} {random.randint(100000000, 999999999)}'

def new_election(db):
    data_attuale = f"{CURRENT_YEAR}-05-18"
    data = date.fromisoformat(data_attuale)
    
    fake = Faker()

    numero_distretti = int(execute_query(db, "SELECT count(*) from distretti")[0][0])
    codice_imperiale = int(execute_query(db, "SELECT max(codiceImperiale) from consoli")[0][0]) + 1
    codice_distretto = int(execute_query(db, "SELECT min(codiceDistretto) from distretti")[0][0])

    for i in range (numero_distretti):
        name = fake.name().split(' ')
        delta = random.randint(-365*80, -165*30)
        new_console = (codice_imperiale, name[0], name[1], data + timedelta(days = delta), data, contatto_random(), codice_distretto)
        execute_query(db, "CALL inserisciConsole(%s, %s, %s, %s, %s, %s, %s)", new_console)
        codice_distretto = codice_distretto + 1
        codice_imperiale = codice_imperiale + 1

Paghiamo casualmente alcuni tributi. Utilizziamo un prepared statement che ci trova consoli compatibili all'occorrenza per semplicità.

In [ ]:
def paga_tributi(db):
    data_attuale = f"{CURRENT_YEAR}-07-20"
    data = date.fromisoformat(data_attuale)

    query_tributi = "select tributi.numeroTributo, tributi.codiceDistretto, distretti.codiceConsole, distretti.CodiceRegione from tributi join distretti using (codiceDistretto) where status = 'non pagato';"
    tributi = execute_query(db, query_tributi)
    
    for t in tributi:
        query_testimoni = "select consoli.codiceImperiale from consoli join distretti on consoli.codiceImperiale = distretti.codiceConsole join regioni using (codiceRegione) where distretti.codiceConsole != %s and regioni.codiceRegione != %s limit 2;"
        testimoni = execute_query(db, query_testimoni, (t[2], t[3]))
        if len (testimoni) < 2 or testimoni == None:
            continue
        payments = random.uniform(0, 1)
        if payments < 0.9:
            query_pagamenti = "insert into pagamenti (DataDiPagamento, NumeroTributo, CodiceConsolePagante, CodiceConsoleTestimone1, CodiceConsoleTestimone2) values (%s, %s, %s, %s, %s)"
            execute_query(db, query_pagamenti, (data, t[0], t[2], testimoni[0][0], testimoni[1][0]))
    return 0

Aggiorniamo casualmente tutti i mondi insieme così da modificare i tributi calcolati.

In [ ]:
def aggiorna_mondi(db):
    delta = random.uniform(0.5, 1.5)
    query = "UPDATE mondi set PIL = PIL*%s"
    execute_query(db, query, (delta, ))
    delta = random.uniform(0.9, 1.1)
    query = "UPDATE mondi SET abitanti = abitanti*%s"
    execute_query(db, query, (delta, ))
    delta = random.uniform(0.95, 1.05)
    query = "UPDATE mondi SET superficieproduttiva = superficieproduttiva*%s"
    execute_query(db, query, (delta, ))
    return 0

**Operazione A**

*Visualizzare i tributi dovuti da ogni Distretto in tempo reale, nonché da ogni Regione per poter comprendere in quali zone esercitare maggiore pressione*

---

L'operazione A è un'operazione senza parametri, dunque sarà sufficiente eseguire una semplice query. Tuttavia diamo la possibilità di scegliere se eseguire l'analisi per Distretto o l'analisi per Regione.

In [ ]:
def operazioneA(db):
    choose = input("Desidera visualizzare i debiti dei distretti o delle regioni? (d/r) ")
    while choose != 'r' and choose != 'd':
        choose = input("Desidera visualizzare i debiti dei distretti o delle regioni? (d/r) ")
    if choose == 'd':
        query = "select sum(if(status = 'non pagato', ammontare, 0)) as debito, codiceDistretto from tributi group by codiceDistretto order by debito; "
        res = execute_query(db, query)
        for x in res:
            print(f"debito : {x[0]} A - distretto : {x[1]}")
    else:
        query = "select sum(if(status = 'non pagato', ammontare, 0)) as debito, distretti.codiceRegione from tributi join Distretti on tributi.codiceDistretto = distretti.CodiceDistretto group by distretti.codiceRegione order by debito;"
        res = execute_query(db, query)
        for x in res:
            print(f"debito : {x[0]} A - regione : {x[1]}")

**Operazione B**

*Visualizzare i 5 Distretti che hanno contribuito maggiormente in un determinato periodo*

---

L'operazione B è un'operazione parametrizzata un po' delicata in quanto gestire le date può essere difficile visto che Python non è tipicizzato. Nessun problema: sfruttiamo la libreria datetime e accettiamo solo date nell'ISOformat YYYY-MM-DD.

In [ ]:
def operazioneB(db):
    query = "select sum(if(status = 'pagato', ammontare, 0)) as contributo, codiceDistretto from tributi join pagamenti on tributi.numeroTributo = pagamenti.numeroTributo where dataDiPagamento between %s and %s group by codiceDistretto order by contributo desc limit 5;"
    data_inizio = input("Inserisci data di inizio analisi contributi nella forma YYYY-MM-DD")
    try:
        data_inizio = date.fromisoformat(data_inizio)
    except Error as e:
        print("Data non valida")
        return 1
    data_fine = input("Inserisci data di fine analisi contributi nella forma YYYY-MM-DD")
    try:
        data_fine = date.fromisoformat(data_fine)
    except Error as e:
        print("Data non valida")
        return 1
    if data_inizio > data_fine:
        print("Data_inizio maggiore di Data_fine, effettueremo uno swap nell'ordine.")
        t = data_fine
        data_fine = data_inizio
        data_inizio = t
    res = execute_query(db, query, (data_inizio, data_fine))
    for x in res:
        print(f"contributo : {x[0]} A - distretto : {x[1]}")

**Operazione C**

*Dato un determinato Distretto visualizzare i tributi che deve nella sua moneta Regionale*

---

L'operazione C ha un singolo parametro, il codice del Distretto desiderato. Pensiamo allora a gestire il caso in cui il codice del Distretto inserito non sia valido.

In [ ]:
def operazioneC(db):
    query = "select sum(if(status = 'non pagato', ammontare, 0))*tassodiconversione as Debito, TUCG.Valuta, tributi.codiceDistretto as Distretto from tributi join distretti on tributi.codiceDistretto = distretti.codiceDistretto join regioni on distretti.codiceRegione = regioni.codiceRegione join TUCG on regioni.valuta = TUCG.valuta where tributi.codiceDistretto = %s group by tributi.codiceDistretto;"
    codiceDistretto = input("Inserisci il codice del distretto di cui si desidera conoscere il debito nella sua moneta Regionale. Se desiderate vedere i possibili codici distretto inserite 'all'")
    if codiceDistretto == "all":
        query = "select codiceDistretto from Distretti order by codiceDistretto;"
        res = execute_query(db, query)
        for x in res:
            print(f"Codice Distretto : {x[0]}")
    else:
        try:
            codiceDistretto = int(codiceDistretto)
        except Error:
            print("Codice Distretto nel formato non corretto")
            return 1
        res = execute_query(db, query, (codiceDistretto,))
        if res == None:
            print("Distretto non trovato")
        else:
            for x in res:
                print(f"Debito : {x[0]} - Valuta : {x[1]}")

**Operazione D**

*Visualizzare i 5 Mondi su cui incidono maggiormente i tributi così da poter dare la possibilità di cambiarne la fascia fiscale di appartenenza per abbassare la pressione fiscale (la pressione è vista come rapporto tra tributi e pil)*

---

L'operazione D è probabilmente la più semplice poiché possiamo vederla come la visualizzazione di una semplice statistica.

In [ ]:
def operazioneD(db):
    query = "select (abitanti*indiceprocapite + superficieproduttiva*indiceprometro)/pil as pressione, nome from mondi join TUFF on fasciaFiscale = codiceFascia order by pressione desc limit 5;"
    res = execute_query(db, query)
    for x in res:
        print(f"Pressione : {(x[0]/1000000):.2e} - Mondo : {x[1]}")

**Operazione E**

*Visualizzare per un dato Distretto lo storico dei Consoli che l’hanno governato*

---

L'operazione E ha al suo interno un solo parametro che gestiremo come già abbiamo visto per l'operazione C.

In [ ]:
def operazioneE(db):
    query = "select nome, cognome, dataDiElezione from consoli where codiceDistretto = %s order by DataDiElezione;"
    codiceDistretto = input("Inserisci il codice del distretto di cui si desidera conoscere lo storico dei Consoli. Se desiderate vedere i possibili codici distretto inserite 'all'")
    if codiceDistretto == "all":
        query = "select codiceDistretto from Distretti order by codiceDistretto;"
        res = execute_query(db, query)
        for x in res:
            print(f"Codice Distretto : {x[0]}")
    else:
        try:
            codice = int(codiceDistretto)
        except Error:
            print("Codice Distretto nel formato non corretto")
            return 1
        res = execute_query(db, query, (codice,))
        if res == None:
            print("Distretto non trovato")
        else:
            for x in res:
                print(f"Nome : {x[0]} - Cognome : {x[1]} - DataDiElezione : {x[2]}")

**Console**

---

La console è un semplice terminale con inserimento da tastiera. I comandi possibili sono i seguenti:
1. **V** : Visualizza lista dei comandi
2. **Q** : Esci dalla console
3. **N** : Simula un nuovo anno
4. **A** : Esegui operazione A
5. **B** : Esegui operazione B
6. **C** : Esegui operazione C
7. **D** : Esegui operazione D
8. **E** : Esegui operazione E

In [ ]:
def operazioneV():
    print("V : Visualizza lista dei comandi")
    print("Q : Esci dalla console")
    print("N : Simula un nuovo anno")
    print("A : Esegui operazione A (Situazione Debiti)")
    print("B : Esegui operazione B (Situazione Tributi)")
    print("C : Esegui operazione C (Valuta Regionale)")
    print("D : Esegui operazione D (Situazione Pressione)")
    print("E : Esegui operazione E (Storico Consoli)")

In [ ]:
def delimiter():
    for i in range (10):
        print("-", end =" ")
    print()

In [ ]:
import time 

def console(db):
    print("Salve Imperatore Agostino MDI. \n Questa è la console per la gestione della base di dati da Lei richiesta. \n Questi sono i comandi.")
    delimiter()
    operazioneV()
    delimiter()
    esc = False
    while esc == False:
        choose = input("Prego, inserisca la lettera dell'operazione che desidera eseguire.")
        if choose.upper() not in ["V", "Q", "N", "A", "B", "C", "D", "E"]:
            print("La lettera desiderata non risulta tra le opzioni, la prego di riprovare.")
        if choose.upper() == "V":
            operazioneV()
            delimiter()
        if choose.upper() == "Q":
            esc = True
            print("Arrivederci. ")
            delimiter()
        if choose.upper() == "N":
            one_year_later(db)
            delimiter()
        if choose.upper() == "A":
            operazioneA(db)
            delimiter()
        if choose.upper() == "B":
            operazioneB(db)
            delimiter()
        if choose.upper() == "C":
            operazioneC(db)
            delimiter()
        if choose.upper() == "D":
            operazioneD(db)
            delimiter()
        if choose.upper() == "E":
            operazioneE(db)
            delimiter() 
        
        time.sleep(2)

Ecco dunque che possiamo usare il programma vero e proprio con una semplice chiamata a Console.

In [ ]:
console(mydb)

In [ ]:
mydb.close()